In [347]:
includet(joinpath(ENV["JULIA_DQMC"], "src/measure_framework.jl"))

In [352]:
measxml = "dqmc.meas.xml"
mp = measxml2MeasParams(measxml)

Loading DQMC parameters (.in.xml)
L = 4
SLICES = 100
DELTA_TAU = 0.1
WARMUP = 100
SWEEPS = 100
SAFE_MULT = 10
CHECKERBOARD = TRUE
HOPPINGS = 1.0,0.5,-0.5,-1.0
MU = -0.5
LAMBDA = 0.5
U = 1.0
R = 2.0
C = 3.0
GLOBAL_UPDATES = TRUE
GLOBAL_RATE = 5
BFIELD = TRUE
WRITE_EVERY_NTH = 10
OPDIM = 3


MeasParams
  requested: Array{Symbol}((2,))
  todo: Array{Symbol}((0,))
  meas_infile: String "dqmc.meas.xml"
  outfile: String "dqmc.meas.h5"
  p: Params
  dqmc_outfile: String "dqmc.out.h5"
  overwrite: Bool true
  num_threads: Int64 1
  walltimelimit: DateTime
  save_after: Int64 2
  to: TimerOutput
  confs_iterator_start: Int64 1


In [353]:
mp.requested

2-element Array{Symbol,1}:
 :etpc_minus
 :etpc_plus 

In [354]:
create_todolist!(mp)

2-element Array{Symbol,1}:
 :etpc_minus
 :etpc_plus 

In [359]:
println("\nLoading DQMC results...")

  # Overwrite mode?
  if isfile(mp.outfile) && mp.overwrite
    jldopen(mp.outfile, "r+") do f
      for o in mp.todo
        p = joinpath("obj/", string(o))
        HDF5.has(f.plain, p) && o_delete(f, p)
      end
    end
  end

  greens = nothing; 
  mc = nothing;

  try
    global confs = loadobs_frommemory(mp.dqmc_outfile, "obs/configurations") # TODO: lazy load?
  catch err
    println("Couldn't read configuration data. Probably no configurations yet? Exiting.")
    exit()
  end

  if need_to_load_etgf(mp)
    if "greens" in listobs(mp.dqmc_outfile)
      greens = loadobs_frommemory(mp.dqmc_outfile, "obs/greens")
    else
      error("Couldn't find ETGF in $(mp.dqmc_outfile) although need_to_load_etgf == true.")
      # TODO: Calculate ETGF if it isn't found in dqmc output.
    end
  end

  # prepare dqmc framework if necessary
  if need_to_setup_mc(mp)
    mc = DQMC(mp.p)
    initialize_stack_for_measurements(mc, mp)
  end


  # ------------------- Create list of observables to measure --------------------
  num_confs = length(confs)
  nsweeps = num_confs * mp.p.write_every_nth
  obs = NamedTuple() # list of observables to measure

  # chi_dyn
  if :chi_dyn_symm in mp.todo
    @addobs chi_dyn_symm Array{Float64, 3}
  end

  if :chi_dyn in mp.todo
    @addobs chi_dyn Array{Float64, 3}
  end
  
  # binder
  if :binder in mp.todo
    @addobs m2s Float64
    @addobs m4s Float64
    obs = add(obs, binder = create_or_load_obs("binder", Float64, name="binder"))
  end

  # etpc
  if :etpc_plus in mp.todo
    zero_etpc = zeros(Float64, mp.p.L, mp.p.L)
    @addlightobs etpc_plus zero_etpc
    @addlightobs etpc_minus zero_etpc
  end

  # zfpc
  if :zfpc_plus in mp.todo
    zero_zfpc = zeros(Float64, mp.p.L, mp.p.L)
    @addlightobs zfpc_plus zero_zfpc
    @addlightobs zfpc_minus zero_zfpc
  end

  # etcdc
  if :etcdc_plus in mp.todo
    zero_etcdc = zeros(ComplexF64, mp.p.L, mp.p.L)
    @addlightobs etcdc_plus zero_etcdc
    @addlightobs etcdc_minus zero_etcdc
  end

  # zfcdc
  if :zfcdc_plus in mp.todo
    zero_zfcdc = zeros(ComplexF64, mp.p.L, mp.p.L)
    @addlightobs zfcdc_plus zero_zfcdc
    @addlightobs zfcdc_minus zero_zfcdc
  end

  # sfdensity
  if :sfdensity in mp.todo
    @addlightobs sfdensity Float64
  end

  # # tdgfs
  # if mp.tdgfs
  #   # create zero element for LightObservable
  #   Nflv = mc.p.flv * mc.l.sites
  #   tdgf_size = (Nflv, Nflv, mc.p.slices)
  #   zero_tdgf = zeros(geltype(mc), tdgf_size)
  #   (obs = add(obs, tdgfs_Gt0 = LightObservable(zero_tdgf, name="Time-displaced Green's function G(tau,0) (TDGF Gt0)", alloc=num_confs)))
  #   (obs = add(obs, tdgfs_G0t = LightObservable(zero_tdgf, name="Time-displaced Green's function G(0, tau) (TDGF G0t)", alloc=num_confs)))
  # end



  # --------------------- Measure -------------------------
  print("\nPrepared Observables ");
  println(keys(obs))
  println()
  reset_timer!(mp.to)


Loading DQMC results...

CB = CBAssaad
p.hoppings = "1.0,0.5,-0.5,-1.0"
p.Nhoppings = "none"
p.NNhoppings = "none"
p.lattice_file = "C:\\Users\\carsten\\Desktop\\sciebo\\lattices\\square_L_4_W_4.xml"


Loading lattice with 16 sites
Initializing neighbor-tables
Initializing Peirls phases (Bfield)
Initializing hopping exponentials (Bfield)
Initializing hopping exponentials (Bfield, Checkerboard)
Checkerboard (Bfield) - exact (abs):		0.0024417942043431936
Allocated memory for ETPC measurements.

Prepared Observables (:etpc_plus, :etpc_minus)



 ──────────────────────────────────────────────────────────────────
                           Time                   Allocations      
                   ──────────────────────   ───────────────────────
 Tot / % measured:      565μs / 0.00%           12.9KiB / 0.00%    

 Section   ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────
 ──────────────────────────────────────────────────────────────────

In [360]:
obs

(etpc_plus = nothing, etpc_minus = nothing)

In [361]:
obs[1]

In [326]:
  measure(mp, obs, confs, greens, mc)

Measuring ...
1
2
Intermediate save...
3
4


Progress:  40%|█████████████████                        |  ETA: 0:00:01

OverflowError: OverflowError: The Binning Analysis has exceeded its maximum capacity.

In [302]:
@code_warntype save_obs_objects(mp, obs)

Body::Nothing
1 ─       invoke Main.println("Intermediate save..."::String)
│         (Main.flush)(Main.stdout)
│   %3  = %new(getfield(Main, Symbol("##577#578")){NamedTuple{(:etpc_plus, :etpc_minus),Tuple{LightObservable{Array{Float64,2},3},LightObservable{Array{Float64,2},3}}}}, obs)::getfield(Main, Symbol("##577#578")){NamedTuple{(:etpc_plus, :etpc_minus),Tuple{LightObservable{Array{Float64,2},3},LightObservable{Array{Float64,2},3}}}}
│   %4  = (Base.getfield)(mp, :outfile)::String
│   %5  = (Base.getfield)(mp, :outfile)::String
│   %6  = invoke Base.Filesystem.stat(%5::String)::Base.Filesystem.StatStruct
│   %7  = (Base.getfield)(%6, :mode)::UInt64
│   %8  = (Base.and_int)(%7, 0x000000000000f000)::UInt64
│   %9  = (%8 === 0x0000000000008000)::Bool
└──       goto #3 if not %9
2 ─       goto #4
3 ─       nothing
4 ┄ %13 = φ (#2 => "r+", #3 => "w")::String
│   %14 = Main.jldopen::Core.Compiler.Const(JLD.jldopen, false)
│         invoke JLD.:(#jldopen#14)($(QuoteNode(Base.Iterators.Pai

In [164]:
  measure(mp, obs, confs, greens, mc)

Measuring ...
1
2
3
4
5


 ──────────────────────────────────────────────────────────────────────────────
                                       Time                   Allocations      
                               ──────────────────────   ───────────────────────
       Tot / % measured:            940ms / 2.42%           1.80MiB / 90.3%    

 Section               ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────────────────
 measure loop               1   22.8ms   100%  22.8ms   1.63MiB  100%   1.63MiB
   load etgf                5   8.59ms  37.7%  1.72ms    428KiB  25.7%  85.6KiB
   bosonic                  5   1.30ms  5.73%   261μs   0.96MiB  59.2%   197KiB
     chi                    5   1.19ms  5.25%   239μs   0.96MiB  58.9%   196KiB
     binder                 5   96.5μs  0.42%  19.3μs   1.95KiB  0.12%     400B
   fermionic                5    252μs  1.11%  50.5μs    200KiB  12.0%  40.0KiB
     etpc                   5    163μs 

In [235]:
  export_results(mp, obs, nsweeps)

Calculating errors and exporting...
Done. Exported to dqmc.meas.h5.
